https://www.kaggle.com/jaskaransingh/fake-news-classification-bert-roberta
https://towardsdatascience.com/text-classification-with-hugging-face-transformers-in-tensorflow-2-without-tears-ee50e4f3e7ed

In [19]:
!pip install transformers

In [20]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
# Evaluation
# import pytorch
import torch
from torch import nn
# import tensorflow_text as text
import pandas as pd
# from official.nlp import optimization  # to create AdamW optimizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
#import np_utils
from keras import backend as K
# from focal_loss import BinaryFocalLoss
import matplotlib.pyplot as plt
from tensorflow.keras import activations
# import tensorflow_addons as tfa


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [21]:
BATCH_SIZE = 32
PRETRAINED_MODEL_NAME = "skimai/spanberta-base-cased"
MAX_SEQUENCE_LENGTH = 125
bert_tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
#bert_model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME,num_labels=2) #FOR CLASSIFICATION
bert_model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME,num_labels=1) #FOR REGRESSION

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skimai/spanberta-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Read data
url = 'https://www.fing.edu.uy/inco/grupos/pln/haha/2019/data/haha_2019_train.csv'
url2 = 'https://www.fing.edu.uy/inco/grupos/pln/haha/2019/data/haha_2019_test_gold.csv'
train = pd.read_csv(url)
testG = pd.read_csv(url2)

train = train.loc[train['is_humor'] == 1].copy()
testG = testG.loc[testG['is_humor'] == 1].copy()

In [22]:
x_train, x_dev,y_train, y_dev = train_test_split(train["text"].tolist(),train["funniness_average"].tolist(), test_size=0.2,stratify=train["is_humor"])

x_test = testG["text"].tolist()
y_test = testG["funniness_average"].tolist()



train_encod = bert_tokenizer(x_train,truncation=True,padding=True,max_length=MAX_SEQUENCE_LENGTH) 
val_encod = bert_tokenizer(x_dev,truncation=True,padding=True,max_length=MAX_SEQUENCE_LENGTH)
test_encod = bert_tokenizer(x_test,truncation=True,padding=True,max_length=MAX_SEQUENCE_LENGTH)

class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

train_ds = NewsGroupsDataset(train_encod, y_train)
dev_ds = NewsGroupsDataset(val_encod, y_dev)
test_ds = NewsGroupsDataset(test_encod, y_test)


In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  #acc = accuracy_score(labels, preds)
  #f1 = f1_score(labels,preds)
  mse = mean_squared_error(labels,pred.predictions,squared=False)
  return {
      #'accuracy': acc,
      #'f1_score':f1,
      'rmse': mse
  }

In [24]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # metric_for_best_model = 'f1_score',
    metric_for_best_model = 'rmse',
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [25]:
trainer = Trainer(
    model=bert_model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_ds,         # training dataset
    eval_dataset=dev_ds,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [26]:
trainer.train()

Step,Training Loss,Validation Loss,Rmse
200,0.960100,0.403716,0.635386
400,0.445700,0.458462,0.677099
600,0.408400,0.396559,0.629730
800,0.386300,0.398583,0.631334


TrainOutput(global_step=926, training_loss=0.5262977237577829, metrics={'train_runtime': 625.7971, 'train_samples_per_second': 1.48, 'total_flos': 187904449507500.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 504422400, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -529268736, 'train_mem_gpu_alloc_delta': 1534215680, 'train_mem_cpu_peaked_delta': 529776640, 'train_mem_gpu_peaked_delta': 1774170624})

In [27]:
trainer.evaluate(eval_dataset=test_ds)

{'epoch': 2.0,
 'eval_loss': 0.4812448024749756,
 'eval_mem_cpu_alloc_delta': 28672,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': -21504,
 'eval_mem_gpu_peaked_delta': 146776576,
 'eval_rmse': 0.6937180757522583,
 'eval_runtime': 17.8133,
 'eval_samples_per_second': 131.475}

In [28]:
trainer.save_model('best_model_spanBERTA.hdf5')

RESULTS CLASSIFICATION

```
Step	Training Loss	Validation Loss	Accuracy	F1 Score
200	0.488500	0.423187	0.810000	0.743820
400	0.410900	0.389528	0.823333	0.727156
600	0.400500	0.403867	0.823542	0.781532
800	0.404100	0.359039	0.845833	0.777644
1000	0.380200	0.387769	0.842292	0.766141
1200	0.349100	0.350626	0.853333	0.795587
1400	0.268600	0.373516	0.850417	0.791158
1600	0.257900	0.379301	0.853333	0.809317
1800	0.248600	0.382039	0.849583	0.813822
2000	0.234700	0.370870	0.856458	0.816707
2200	0.261100	0.354002	0.854583	0.806219
2400	0.228700	0.362816	0.858333	0.817008

Test 

'epoch': 2.0,
 'eval_accuracy': 0.8356666666666667,
 'eval_f1_score': 0.7987755102040816,
 'eval_loss': 0.4098193347454071,
 'eval_mem_cpu_alloc_delta': 16384,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 162115584,
 'eval_runtime': 48.4072,
 'eval_samples_per_second': 123.948}

```

REGRESSION


```

Test

```
